# 項目14～16

In [6]:
import random
random.seed(1234)

import logging
from pprint import pprint
from sys import stdout as STDOUT

# Write all output to a temporary directory
import atexit
import gc
import io
import os
import tempfile

TEST_DIR = tempfile.TemporaryDirectory()
atexit.register(TEST_DIR.cleanup)

# Make sure Windows processes exit cleanly
OLD_CWD = os.getcwd()
atexit.register(lambda: os.chdir(OLD_CWD))
os.chdir(TEST_DIR.name)

def close_open_files():
    everything = gc.get_objects()
    for obj in everything:
        if isinstance(obj, io.IOBase):
            obj.close()

atexit.register(close_open_files)

<function __main__.close_open_files()>

## 項目14 key引数を使い複雑な基準でソートする

`list`には様々な基準に基づいて`list`の要素を順序に従って整列するsortメソッドがある。

デフォルトのsortでは、listの内容を要素の自然な順序の昇順に並べる。

例えば、次のコードは整数を小さいものから大きいものに並べる。

In [1]:
# Example 1
numbers = [93, 86, 11, 68, 70]
numbers.sort()
print(numbers)

[11, 68, 70, 86, 93]


sortメソッドは自然な順序のあるほとんどの組み込み型（文字列、浮動小数点数など）で動作する。

ここでオブジェクトに対してどのような動作が起きるか見てみよう。

例えば、建築現場で使う様々な工具を表すクラスを、インスタンスを出力できるよう`__repr__`メソッド（項目75参照）を含めて定義する。

In [3]:
# Example 2
class Tool:
    def __init__(self, name, weight):
        self.name = name
        self.weight = weight

    def __repr__(self):
        return f'Tool({self.name!r}, {self.weight})'

tools = [
    Tool('level', 3.5),
    Tool('hammer', 1.25),
    Tool('screwdriver', 0.5),
    Tool('chisel', 0.25),
]

この型のオブジェクトのソートを行おうとすると、（クラスで定義されていない）比較のための特殊メソッドを`sort`メソッドが呼び出そうとするため失敗する。

In [7]:
# Example 3
try:
    tools.sort()
except:
    logging.exception('Expected')
else:
    assert False

ERROR:root:Expected
Traceback (most recent call last):
  File "C:\Users\su10_\AppData\Local\Temp\ipykernel_28784\3010603424.py", line 3, in <cell line: 2>
    tools.sort()
TypeError: '<' not supported between instances of 'Tool' and 'Tool'


クラスに（整数のような）自然な順序がない場合には、必要な特殊メソッド（項目73参照）を定義して、追加のパラメータがなくても`sort`が動作するようにできる。

しかし、より一般的な場合には、オブジェクトで複数の順序付けをサポートする必要があり、自然な順序の定義だけでは意味がない。

しばしば、オブジェクトの属性値によってソートする場合がある。このユースケースをサポートするために、`sort`メソッドには関数を引数とする`key`パラメータがある。

`key`関数には単一引数としてソートされるリストの要素が渡される。戻り値は、ソートのために使用する比較可能な（自然な順序を持つ）値でなければならない。

次のコードでは、`Tool`オブジェクトのリストを名前の英字淳にソートできる`key`パラメータのための関数をlambdaキーワードを用いて定義する。

In [9]:
# Example 4
print('Unsorted:', repr(tools))
tools.sort(key=lambda x: x.name)
print('\nSorted:  ', tools)

Unsorted: [Tool('level', 3.5), Tool('hammer', 1.25), Tool('screwdriver', 0.5), Tool('chisel', 0.25)]

Sorted:   [Tool('chisel', 0.25), Tool('hammer', 1.25), Tool('level', 3.5), Tool('screwdriver', 0.5)]


重さでソートするラムダ関数も定義でき、`sort`メソッドに`key`パラメータとして渡すことができる。

In [10]:
# Example 5
tools.sort(key=lambda x: x.weight)
print('By weight:', tools)

By weight: [Tool('chisel', 0.25), Tool('screwdriver', 0.5), Tool('hammer', 1.25), Tool('level', 3.5)]


`key`パラメータとして渡されるラムダ関数の中で要素の属性に、（シーケンス、タプル、辞書では）要素のインデックス、あるいは、他の式を用いてアクセスすることができる。

文字列のような基本型では、`key`関数を使ってソートの前に値の変換ができる。

例えば次のコードでは、通常のソートでは大文字が小文字の前に来るところを、
大文字小文字を無視した英字順なるよう、`list`の中の要素の名前に`lower`メソッドを適用している。

In [11]:
# Example 6
places = ['home', 'work', 'New York', 'Paris']
places.sort()
print('Case sensitive:  ', places)
places.sort(key=lambda x: x.lower())
print('Case insensitive:', places)

Case sensitive:   ['New York', 'Paris', 'home', 'work']
Case insensitive: ['home', 'New York', 'Paris', 'work']


In [ ]:



# Example 7
power_tools = [
    Tool('drill', 4),
    Tool('circular saw', 5),
    Tool('jackhammer', 40),
    Tool('sander', 4),
]


# Example 8
saw = (5, 'circular saw')
jackhammer = (40, 'jackhammer')
assert not (jackhammer < saw)  # Matches expectations


# Example 9
drill = (4, 'drill')
sander = (4, 'sander')
assert drill[0] == sander[0]  # Same weight
assert drill[1] < sander[1]   # Alphabetically less
assert drill < sander         # Thus, drill comes first


# Example 10
power_tools.sort(key=lambda x: (x.weight, x.name))
print(power_tools)


# Example 11
power_tools.sort(key=lambda x: (x.weight, x.name),
                 reverse=True)  # Makes all criteria descending
print(power_tools)


# Example 12
power_tools.sort(key=lambda x: (-x.weight, x.name))
print(power_tools)


# Example 13
try:
    power_tools.sort(key=lambda x: (x.weight, -x.name),
                     reverse=True)
except:
    logging.exception('Expected')
else:
    assert False


# Example 14
power_tools.sort(key=lambda x: x.name)   # Name ascending

power_tools.sort(key=lambda x: x.weight, # Weight descending
                 reverse=True)

print(power_tools)


# Example 15
power_tools.sort(key=lambda x: x.name)
print(power_tools)


# Example 16
power_tools.sort(key=lambda x: x.weight,
                 reverse=True)
print(power_tools)
